In [12]:
import numpy as np
from pathlib import Path
import pandas as pd
# import spiketimes
from spiketimes.df.binning import which_bin
from spiketimes.df.alignment import align_around


In [36]:
MICE = [
    {"name": "B51618", "path": Path(r"E:\exports\B51618"), "group": "exp"},
    {"name": "B51622", "path": Path(r"E:\exports\B51622"), "group": "no shock"},
    {"name": "B51629", "path": Path(r"E:\exports\B51629"), "group": "one context"}
]
SHOCK = np.array([240, 300, 360, 420, 480, 540])
CONTEXTS = np.arange(6) * 120
CONTEXT_NAMES = ["safe", "shock"]

In [62]:
def load_day3_df(p):
    return pd.read_csv(p / "day3-noon" / "traces_updated.csv")

def load_day4_df(p):
    return (
        pd.read_csv(p / "day4-test1" / "traces_updated.csv")
        .pipe(lambda x: 
            which_bin(x, CONTEXTS, spiketimes_col="time").assign(context= lambda x: x.bin_idx.apply(lambda x: CONTEXT_NAMES[int(x % 2)]))
        )
    )

def load(loader):
    dfs = []
    for d in MICE:
        dfs.append(
            loader(d["path"]).assign(name=d["name"], group=d["group"])
        )
    return pd.concat(dfs)


In [63]:
# day3

df_d3 = load(loader=load_day3_df)
align_around(df_d3, data_colname="time", events=SHOCK, t_before=10, max_latency=10).dropna().head(4)

,time,session_cell_id,value,cell_id,local_cellset_index,name,group,aligned
2302,230.020444,0,73.06292,0,0,B51618,exp,-9.979556
2303,230.120366,0,79.84438,0,0,B51618,exp,-9.879634
2304,230.220288,0,90.00584,0,0,B51618,exp,-9.779712
2305,230.320210,0,91.78522,0,0,B51618,exp,-9.679790


In [64]:
# day4
df_d4 = load(loader=load_day4_df)

In [66]:
df_d4

,time,session_cell_id,value,cell_id,local_cellset_index,bin_idx,bin_values,context,name,group
0,0.000000,0,23.30407,22,1,0.0,0.0,safe,B51618,exp
1,0.099922,0,32.12091,22,1,0.0,0.0,safe,B51618,exp
2,0.199844,0,37.35959,22,1,0.0,0.0,safe,B51618,exp
3,0.299766,0,37.64541,22,1,0.0,0.0,safe,B51618,exp
4,0.399688,0,38.37547,22,1,0.0,0.0,safe,B51618,exp
...,...,...,...,...,...,...,...,...,...,...
319831,725.833408,101,16.85463,65,1,5.0,600.0,shock,B51629,one context
319832,725.933330,101,17.17459,65,1,5.0,600.0,shock,B51629,one context
319833,726.033252,101,15.80440,65,1,5.0,600.0,shock,B51629,one context
319834,726.133174,101,16.53741,65,1,5.0,600.0,shock,B51629,one context
